# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk
import time

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

nltk.download(['punkt', 'wordnet', 'stopwords', 'averaged_perceptron_tagger'])

import warnings

warnings.simplefilter('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database

engine = create_engine('sqlite:///Messages_P.db')
df = pd.read_sql("SELECT * FROM Messages_P", engine)

X = df.filter(items=['id', 'message', 'original', 'genre'])
Y = df.drop(['id', 'message', 'original', 'genre', 'child_alone'], axis=1)#'child_alone' has no responses
#Mapping the '2' values in 'related' to '1' - because I consider them as a response (that is, '1')
Y['related']=Y['related'].map(lambda x: 1 if x == 2 else x)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
print(X[:2].values.tolist())
#need sent_tokenize to parse X.

[[2, 'Weather update - a cold front from Cuba that could pass over Haiti', 'Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi', 'direct'], [7, 'Is the Hurricane over or is it not over', 'Cyclone nan fini osinon li pa fini', 'direct']]


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(w).lower().strip() for w in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier())
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train['message'], y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred_test = pipeline.predict(X_test['message'])
y_pred_train = pipeline.predict(X_train['message'])
print(classification_report(y_test.values, y_pred_test, target_names=Y.columns.values))
print('---------- °°°°°________________ °°°°° ----------°---------- °°°°°________________ °°°°° ----------')
print('\n',classification_report(y_train.values, y_pred_train, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.91      0.88      5041
               request       0.85      0.37      0.52      1155
                 offer       0.00      0.00      0.00        21
           aid_related       0.78      0.43      0.56      2760
          medical_help       0.50      0.01      0.01       550
      medical_products       0.80      0.04      0.07       340
     search_and_rescue       0.71      0.03      0.05       177
              security       0.00      0.00      0.00       110
              military       0.80      0.02      0.03       227
                 water       0.85      0.12      0.20       449
                  food       0.87      0.26      0.40       746
               shelter       0.84      0.14      0.24       593
              clothing       1.00      0.04      0.07       105
                 money       0.50      0.01      0.01       161
        missing_people       0.00      

In [8]:
# Calculation the proportion of each column that have label == 1
Y.sum()/len(Y)

related                   0.766478
request                   0.170659
offer                     0.004501
aid_related               0.414251
medical_help              0.079493
medical_products          0.050084
search_and_rescue         0.027617
security                  0.017966
military                  0.032804
water                     0.063778
food                      0.111497
shelter                   0.088267
clothing                  0.015449
money                     0.023039
missing_people            0.011367
refugees                  0.033377
death                     0.045545
other_aid                 0.131446
infrastructure_related    0.065037
transport                 0.045812
buildings                 0.050847
electricity               0.020293
tools                     0.006065
hospitals                 0.010795
shops                     0.004577
aid_centers               0.011787
other_infrastructure      0.043904
weather_related           0.278341
floods              

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters = {'clf__max_depth': [10, 20, None],
              'clf__min_samples_leaf': [1, 2, 4],
              'clf__min_samples_split': [2, 5, 10],
              'clf__n_estimators': [10, 20, 40]}

In [10]:
a = y_pred_train
print(a)


[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]


In [11]:
cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', verbose=1, n_jobs=-1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
print(parameters)
t0 = time.time()

cv.fit(X_train['message'], y_train)

labels = np.unique(y_pred_test)
accuracy = (y_pred_test == y_test).mean()
best_parameters = cv.best_estimator_.get_params()

print("Labels:", labels)
print("Accuracy:", accuracy)
print("\nBest Parameters:", best_parameters)


for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

cv.fit(X_train['message'], y_train)

Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'clf__max_depth': [10, 20, None], 'clf__min_samples_leaf': [1, 2, 4], 'clf__min_samples_split': [2, 5, 10], 'clf__n_estimators': [10, 20, 40]}
Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed: 88.9min finished


Labels: [ 0.  1.]
Accuracy: related                   0.803631
request                   0.877327
offer                     0.996796
aid_related               0.711169
medical_help              0.916082
medical_products          0.949496
search_and_rescue         0.973451
security                  0.983216
military                  0.965822
water                     0.938053
food                      0.911657
shelter                   0.919591
clothing                  0.984590
money                     0.975435
missing_people            0.989319
refugees                  0.965670
death                     0.956363
other_aid                 0.861764
infrastructure_related    0.939884
transport                 0.956668
buildings                 0.948428
electricity               0.982453
tools                     0.993744
hospitals                 0.991913
shops                     0.996033
aid_centers               0.987641
other_infrastructure      0.959872
weather_related           0

KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
y_pred_test = cv.predict(X_test['message'])
y_pred_train = cv.predict(X_train['message'])
print(classification_report(y_test.values, y_pred_test, target_names=Y.columns.values))
print('---------- °°°°°________________ °°°°° ----------°---------- °°°°°________________ °°°°° ----------')
print('\n',classification_report(y_train.values, y_pred_train, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.96      0.89      5041
               request       0.88      0.42      0.57      1155
                 offer       0.00      0.00      0.00        21
           aid_related       0.82      0.52      0.63      2760
          medical_help       0.60      0.01      0.01       550
      medical_products       0.70      0.02      0.04       340
     search_and_rescue       1.00      0.01      0.02       177
              security       0.00      0.00      0.00       110
              military       0.67      0.01      0.02       227
                 water       0.95      0.13      0.23       449
                  food       0.91      0.24      0.38       746
               shelter       0.91      0.08      0.15       593
              clothing       1.00      0.02      0.04       105
                 money       0.00      0.00      0.00       161
        missing_people       0.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
cv.best_params_

{'clf__max_depth': None,
 'clf__min_samples_leaf': 1,
 'clf__min_samples_split': 5,
 'clf__n_estimators': 40}

### 9. Export your model as a pickle file

In [15]:
m = pickle.dumps('clf')


In [16]:
print(m)

b'\x80\x03X\x03\x00\x00\x00clfq\x00.'


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.